#### **📌 ¿Qué es Swarm?**

Swarm es un modelo de equipo de agentes en el que **los agentes pueden delegar tareas a otros agentes** en función de sus capacidades. **No hay un orquestador central** como en `SelectorGroupChat`; en su lugar, los agentes toman decisiones locales sobre cuándo y a quién delegar las tareas.

#### **🛠️ Características clave:**

- **Delegación de tareas entre agentes.**
- **Compartición de contexto:** Todos los agentes ven el historial de mensajes.
- **Sin orquestador central:** Cada agente decide si delega o no.
- **Basado en HandoffMessages:** Un agente transfiere la tarea a otro mediante un mensaje especial.
- **Evita ejecución paralela descontrolada:** Se recomienda desactivar `parallel_tool_calls` si el modelo lo permite.

---

### **🚀 ¿Cómo funciona?**

1. **Los agentes pueden generar un `HandoffMessage`** para transferir la tarea a otro agente.
2. **El primer agente inicia la tarea y decide si debe delegarla**.
3. **Si se genera un `HandoffMessage`, el agente receptor toma el control** y continúa con la misma conversación.
4. **El proceso continúa hasta que se cumple una condición de finalización**, como una palabra clave (`TERMINATE`) o cuando el usuario debe responder.

---

## **🛫 Ejemplo 1: Atención al Cliente con Swarm**

Este ejemplo simula un sistema de **reembolso de vuelos** con dos agentes:

- **`TravelAgent`**: Maneja consultas generales sobre viajes y coordina reembolsos.
- **`FlightsRefunder`**: Procesa reembolsos de vuelos usando la herramienta `refund_flight()`.
- **El usuario puede interactuar directamente** cuando un agente le transfiere la conversación.

### **🔹 Flujo del sistema**

1. **El `TravelAgent` inicia la conversación** y analiza la solicitud del usuario.
2. **Si la solicitud es un reembolso, transfiere la tarea al `FlightsRefunder`**.
3. **El `FlightsRefunder` solicita el número de vuelo** y transfiere la conversación al usuario.
4. **El usuario responde con el número de vuelo**, el `FlightsRefunder` lo procesa y transfiere la conversación de vuelta al `TravelAgent`.
5. **El `TravelAgent` confirma el reembolso y finaliza con `TERMINATE`**.

### **📝 Código de ejemplo**

In [ ]:
from typing import Any, Dict, List
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 🔹 Herramienta para procesar reembolsos
def refund_flight(flight_id: str) -> str:
    """Simula el reembolso de un vuelo"""
    return f"Flight {flight_id} refunded"

# 🔹 Configuración del cliente de modelo
model_client = OpenAIChatCompletionClient(model="gpt-4o")

# 🔹 Definición de agentes
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""
    You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, first send your message, then handoff to the user.
    Use TERMINATE when the travel planning is complete.
    """,
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""
    You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    Use the refund_flight tool to process refunds.
    If you need information from the user, first send your message, then handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.
    """,
)

# 🔹 Condiciones de terminación
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")

# 🔹 Creación del equipo Swarm
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

# 🔹 Tarea inicial
task = "I need to refund my flight, the id is 98123D."

async def run_team_stream():
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    # 🔹 Si el agente transfiere al usuario, espera entrada manual
    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")
        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]

# 🔹 Ejecutar el sistema
await run_team_stream()


---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
[FunctionCall(id='call_HTBhcEi8Jze2M5SgwYpZNnMq', arguments='{}', name='transfer_to_flights_refunder')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='call_HTBhcEi8Jze2M5SgwYpZNnMq')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
I can help you refund your flight. Could you please provide me with your flight reference number?
---------- flights_refunder ----------
[FunctionCall(id='call_jJNjSlVIY4KprSW25XbHN9wj', arguments='{}', name='transfer_to_user')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', call_id='call_jJNjSlVIY4KprSW25XbHN9wj')]
---------- flights_refunder ----------
Transf

---

## **📈 Ejemplo 2: Investigación de Mercado con Swarm**

Este sistema simula un equipo autónomo para **investigar el mercado de acciones** con cuatro agentes:

- **`Planner`**: Coordina la investigación y delega tareas.
- **`FinancialAnalyst`**: Obtiene y analiza datos financieros de una acción.
- **`NewsAnalyst`**: Busca noticias recientes sobre la empresa.
- **`Writer`**: Redacta un informe final con los hallazgos.

### **🔹 Flujo del sistema**

1. **El `Planner` inicia la investigación** y delega tareas a los agentes especializados.
2. **Cada agente trabaja de manera independiente** y comparte sus resultados en el historial.
3. **Cuando terminan, devuelven el control al `Planner`**.
4. **El `Writer` genera un informe final y el `Planner` termina con `TERMINATE`**.

### **📝 Código de ejemplo**

In [ ]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Simula la obtención de datos de mercado"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}

async def get_news(query: str) -> List[Dict[str, str]]:
    """Simula la obtención de noticias sobre una empresa"""
    return [
        {"title": "Tesla Expands Cybertruck Production", "summary": "Tesla aumenta producción en Gigafactory Texas."},
        {"title": "Model Y Dominates Global EV Sales", "summary": "El Model Y es el EV más vendido mundialmente."}
    ]

model_client = OpenAIChatCompletionClient(model="gpt-4o")

planner = AssistantAgent(
    "planner", model_client=model_client, handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="You are a research planner. Delegate tasks and compile findings. Use TERMINATE when complete."
)

financial_analyst = AssistantAgent(
    "financial_analyst", model_client=model_client, handoffs=["planner"], tools=[get_stock_data],
    system_message="Analyze stock data using get_stock_data tool. Always return to the planner."
)

news_analyst = AssistantAgent(
    "news_analyst", model_client=model_client, handoffs=["planner"], tools=[get_news],
    system_message="Gather news using get_news tool. Always return to the planner."
)

writer = AssistantAgent(
    "writer", model_client=model_client, handoffs=["planner"],
    system_message="Compile findings into a final report. Always return to the planner."
)

termination = TextMentionTermination("TERMINATE")

research_team = Swarm([planner, financial_analyst, news_analyst, writer], termination_condition=termination)

task = "Conduct market research for TSLA stock"
await Console(research_team.run_stream(task=task))



---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
[FunctionCall(id='call_ZV4D74yBsXBsVhpqXdblQRyS', arguments='{}', name='transfer_to_flights_refunder')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='call_ZV4D74yBsXBsVhpqXdblQRyS')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
Please provide me with the flight reference number that you would like to refund.
---------- flights_refunder ----------
[FunctionCall(id='call_mA2SrXJHI4cZfLI5iMhGk1F2', arguments='{}', name='transfer_to_user')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', call_id='call_mA2SrXJHI4cZfLI5iMhGk1F2')]
---------- flights_refunder ----------
Transferred to user, a